# Assignment 5: Implementing Load-Balanced Custom Tunneling in P4

There are 2 parts to this assignment:

1. Part A: Implement Custom Tunneling Protocol (10 points)
2. Part B: Implement ECMP Load Balancing (Bonus - 5 points)

In Part A, you will implement a custom tunneling protocol in the given topology with 4 switches and 2 hosts. The goal is to encapsulate packets with a custom tunnel header and forward them based on the destination ID field in the custom header.

Part B is a bonus section that focuses on extending the functionality of the P4 program to implement an Equal-Cost Multi-Path (ECMP) load balancer on switch S1 using 5-tuple hashing. The load balancer will distribute the traffic from host h1 to host h2 between switches S2 and S3 based on the hash result.

> Note: Part A is the core requirement for this assignment, and completing it successfully is the primary objective. Part B is an optional bonus section that allows you to showcase your skills and gain additional recognition for implementing ECMP load balancing.

In [ ]:
# Your hawk username, for example "jdoe"
student_id = ""
# To use verbose output 
# (prints most commands as they are run, for debugging help)
verbose = 1

## Configuration & Setup

In [ ]:
# Read configuration file
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network
config_name = f"{student_id}_assignment5.txt"
config_file = open(config_name, 'r')
configs = [line.rstrip('\n') for line in config_file]

# Read subnet config
subnet = IPv4Network(configs[0])

# Read host addresses
h1_ipv4 = IPv4Address(configs[1])
h2_ipv4 = IPv4Address(configs[2])

# Read switch MACs
s1_macs = configs[3].split(',')
s2_macs = configs[4].split(',')
s3_macs = configs[5].split(',')
s4_macs = configs[6].split(',')

# Read host MACs
h1_mac = configs[7]
h2_mac = configs[8]

# Read IPv4 switch network addresses
s1_ipv4 = configs[9]
s2_ipv4 = configs[10]
s3_ipv4 = configs[11]
s4_ipv4 = configs[12]

print("✅ Reading config... DONE!")


## Create topology

<img src="./images/topology_diagram.png" width="700px"/>

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
slice_name = f"{student_id}_assignment5"
slice = fablib.new_slice(name=slice_name)
host_cores = 2
host_ram = 2
host_disk = 10
switch_cores = 4
switch_ram = 8
switch_disk = 10
image = "default_ubuntu_20"
site="WASH"

# Build Subnet
try:
    # Hosts
    h1 = slice.add_node(name="h1", cores=host_cores, ram=host_ram, disk=host_disk, image=image, site=site)
    h1_iface = h1.add_component(model="NIC_Basic").get_interfaces()[0]
    h2 = slice.add_node(name="h2", cores=host_cores, ram=host_ram, disk=host_disk, image=image, site=site)
    h2_iface = h2.add_component(model="NIC_Basic").get_interfaces()[0]

    # Switches
    s1 = slice.add_node(name="s1", cores=switch_cores, ram=switch_ram, disk=switch_disk, image=image, site=site)
    s1_iface0 = s1.add_component(model="NIC_Basic", name='s1p0').get_interfaces()[0]
    s1_iface1 = s1.add_component(model="NIC_Basic", name='s1p1').get_interfaces()[0]
    s1_iface2 = s1.add_component(model="NIC_Basic", name='s1p2').get_interfaces()[0]

    s2 = slice.add_node(name="s2", cores=switch_cores, ram=switch_ram, disk=switch_disk, image=image, site=site)
    s2_iface0 = s2.add_component(model="NIC_Basic", name='s2p0').get_interfaces()[0]
    s2_iface1 = s2.add_component(model="NIC_Basic", name='s2p1').get_interfaces()[0]

    s3 = slice.add_node(name="s3", cores=switch_cores, ram=switch_ram, disk=switch_disk, image=image, site=site)
    s3_iface0 = s3.add_component(model="NIC_Basic", name='s3p0').get_interfaces()[0]
    s3_iface1 = s3.add_component(model="NIC_Basic", name='s3p1').get_interfaces()[0]

    s4 = slice.add_node(name="s4", cores=switch_cores, ram=switch_ram, disk=switch_disk, image=image, site=site)
    s4_iface0 = s4.add_component(model="NIC_Basic", name='s4p0').get_interfaces()[0]
    s4_iface1 = s4.add_component(model="NIC_Basic", name='s4p1').get_interfaces()[0]
    s4_iface2 = s4.add_component(model="NIC_Basic", name='s4p2').get_interfaces()[0]

    # Connect hosts to switches
    s1h1 = slice.add_l2network(name='s1h1', interfaces=[s1_iface0, h1_iface])
    s4h2 = slice.add_l2network(name='s4h2', interfaces=[s4_iface2, h2_iface])

    # Connect switches
    s1s2 = slice.add_l2network(name='s1s2', interfaces=[s1_iface1, s2_iface0])
    s1s3 = slice.add_l2network(name='s1s3', interfaces=[s1_iface2, s3_iface0])
    s2s4 = slice.add_l2network(name='s2s4', interfaces=[s2_iface1, s4_iface0])
    s3s4 = slice.add_l2network(name='s3s4', interfaces=[s3_iface1, s4_iface1])

    slice.submit()

    print("✅ DONE! Topology created successfully")
except Exception as e:
    print(f"Error: {e}")

s1 = slice.get_node(name='s1')
s2 = slice.get_node(name='s2')
s3 = slice.get_node(name='s3')
s4 = slice.get_node(name='s4')
h1 = slice.get_node(name='h1')
h2 = slice.get_node(name='h2')

switches = [s1, s2, s3, s4]
hosts = [h1, h2]

## Internet Connectivity Test

Before proceeding with the installation of required packages and tools on the nodes, it's crucial to ensure that each node has proper internet connectivity. In some rare instances, due to slice configuration issues on FABRIC, a node might lack internet access, which could lead to difficulties in installing the necessary tools. This issue may go unnoticed until the end of our testing process, causing unnecessary delays and complications.

In [ ]:
# Test ping connectivity to internet (iit.edu) for each switch and host
# If the nodes can't ping to the the external n/w then packages can't be installed
try:
    print("Testing ping connectivity to iit.edu...")
    for switch in switches:
        stdout, stderr = switch.execute("ping -c 1 iit.edu", quiet=True)
        print(f"{switch.get_name()} - Ping {'successful' if '0% packet loss' in stdout else 'failed'}")
    
    for host in hosts:
        stdout, stderr = host.execute("ping -c 1 iit.edu", quiet=True)
        print(f"{host.get_name()} - Ping {'successful' if '0% packet loss' in stdout else 'failed'}")
    
    print("Ping connectivity test completed.")
except Exception as e:
    print(f"Error during ping connectivity test: {e}")

## Delete Slice 🛑

If the ping test failed on atleast one node OR there was a node failure during slice provisioning, delete the slice and create a new one.

> Note: Don't forget to comment this code back, else it could lead accidental deletion of the slice!

In [ ]:
# UNCOMMENT the line below and execute this cell, then try again. 
# slice.delete()

# Install BMv2, net-tools and scapy

> Note this will take an ample amount of time finish, please do NOT interrupt/reload/restart

In [ ]:
# Install scripts and turn up interfaces
try:
    install_bmv2 = '. /etc/os-release && echo "deb https://download.opensuse.org/repositories/home:/p4lang/xUbuntu_${VERSION_ID}/ /" | sudo tee /etc/apt/sources.list.d/home:p4lang.list && curl -L "https://download.opensuse.org/repositories/home:/p4lang/xUbuntu_${VERSION_ID}/Release.key" | sudo apt-key add - && sudo apt-get update && sudo apt install -y p4lang-p4c'
    install_net_tools = 'sudo apt-get install -y net-tools'
    install_scapy = 'sudo apt-get update && sudo apt-get install -y python3-scapy'
    install_tshark = 'sudo DEBIAN_FRONTEND=noninteractive apt install tshark -y'
    for switch in switches:
        stdout, stderr = switch.execute(install_bmv2, quiet=True)
        stdout, stderr = switch.execute(install_net_tools, quiet=True)
        stdout, stderr = switch.execute(install_tshark, quiet=True)
        if type(ip_address(switch.get_management_ip())) is IPv6Address:
            switch.execute("sudo sh -c 'echo nameserver 2a00:1098:2c::1 >> /etc/resolv.conf' && sudo sh -c 'echo nameserver 2a01:4f8:c2c:123f::1 >> /etc/resolv.conf' && sudo sh -c 'echo nameserver 2a00:1098:2b::1 >> /etc/resolv.conf'")
    for host in hosts:
        stdout, stderr = host.execute(f'{install_net_tools} && {install_scapy} && {install_tshark}', quiet=True)
        if type(ip_address(host.get_management_ip())) is IPv6Address:
            host.execute("sudo sh -c 'echo nameserver 2a00:1098:2c::1 >> /etc/resolv.conf' && sudo sh -c 'echo nameserver 2a01:4f8:c2c:123f::1 >> /etc/resolv.conf' && sudo sh -c 'echo nameserver 2a00:1098:2b::1 >> /etc/resolv.conf'")
    
    # Turn up interfaces
    s1_iface0 = s1.get_interface(network_name='s1s2')
    s1_iface0_name = s1_iface0.get_device_name()
    stdout, stderr = s1.execute(f'sudo ip link set dev {s1_iface0_name} up', quiet=True)
    s1_iface1 = s1.get_interface(network_name='s1h1')
    s1_iface1_name = s1_iface1.get_device_name()
    stdout, stderr = s1.execute(f'sudo ip link set dev {s1_iface1_name} up', quiet=True)
    s1_iface2 = s1.get_interface(network_name='s1s3')
    s1_iface2_name = s1_iface2.get_device_name()
    stdout, stderr = s1.execute(f'sudo ip link set dev {s1_iface2_name} up', quiet=True)
    h1_iface = h1.get_interface(network_name='s1h1')
    h1_iface_name = h1_iface.get_device_name()
    stdout, stderr = h1.execute(f'sudo ip link set dev {h1_iface_name} up', quiet=True)
    
    s2_iface0 = s2.get_interface(network_name='s1s2')
    s2_iface0_name = s2_iface0.get_device_name()
    stdout, stderr = s2.execute(f'sudo ip link set dev {s2_iface0_name} up', quiet=True)
    s2_iface1 = s2.get_interface(network_name='s2s4')
    s2_iface1_name = s2_iface1.get_device_name()
    stdout, stderr = s2.execute(f'sudo ip link set dev {s2_iface1_name} up', quiet=True)
    
    s3_iface0 = s3.get_interface(network_name='s1s3')
    s3_iface0_name = s3_iface0.get_device_name()
    stdout, stderr = s3.execute(f'sudo ip link set dev {s3_iface0_name} up', quiet=True)
    s3_iface1 = s3.get_interface(network_name='s3s4')
    s3_iface1_name = s3_iface1.get_device_name()
    stdout, stderr = s3.execute(f'sudo ip link set dev {s3_iface1_name} up', quiet=True)
    
    s4_iface0 = s4.get_interface(network_name='s2s4')
    s4_iface0_name = s4_iface0.get_device_name()
    stdout, stderr = s4.execute(f'sudo ip link set dev {s4_iface0_name} up', quiet=True)
    s4_iface1 = s4.get_interface(network_name='s3s4')
    s4_iface1_name = s4_iface1.get_device_name()
    stdout, stderr = s4.execute(f'sudo ip link set dev {s4_iface1_name} up', quiet=True)
    s4_iface2 = s4.get_interface(network_name='s4h2')
    s4_iface2_name = s4_iface2.get_device_name()
    stdout, stderr = s4.execute(f'sudo ip link set dev {s4_iface2_name} up', quiet=True)
    h2_iface = h2.get_interface(network_name='s4h2')
    h2_iface_name = h2_iface.get_device_name()
    stdout, stderr = h2.execute(f'sudo ip link set dev {h2_iface_name} up', quiet=True)
    
    # Enable IPv4 Forwarding on switches
    stdout, stderr = s1.execute("sudo sysctl net.ipv4.ip_forward=1", quiet=True)
    stdout, stderr = s2.execute("sudo sysctl net.ipv4.ip_forward=1", quiet=True)
    stdout, stderr = s3.execute("sudo sysctl net.ipv4.ip_forward=1", quiet=True)
    stdout, stderr = s4.execute("sudo sysctl net.ipv4.ip_forward=1", quiet=True)

    print("✅ DONE! Installation was successful.")
except Exception as e:
    print(f"Error: {e}")

## Configure MAC and IP addresses

In [ ]:
# Configure MAC and IP Addresses
try:
    # MACs
    if verbose: print(f's1: sudo ifconfig {s1_iface0_name} hw ether {s1_macs[0]}')
    stdout, stderr = s1.execute(f'sudo ifconfig {s1_iface0_name} hw ether {s1_macs[0]}', quiet=True)
    if verbose: print(f's1: sudo ifconfig {s1_iface1_name} hw ether {s1_macs[1]}')
    stdout, stderr = s1.execute(f'sudo ifconfig {s1_iface1_name} hw ether {s1_macs[1]}', quiet=True)
    if verbose: print(f's1: sudo ifconfig {s1_iface2_name} hw ether {s1_macs[2]}')
    stdout, stderr = s1.execute(f'sudo ifconfig {s1_iface2_name} hw ether {s1_macs[2]}', quiet=True)
    if verbose: print(f'h1: sudo ifconfig {h1_iface_name} hw ether {h1_mac}')
    stdout, stderr = h1.execute(f'sudo ifconfig {h1_iface_name} hw ether {h1_mac}', quiet=True)
    if verbose: print(f's2: sudo ifconfig {s2_iface0_name} hw ether {s2_macs[0]}')
    stdout, stderr = s2.execute(f'sudo ifconfig {s2_iface0_name} hw ether {s2_macs[0]}', quiet=True)
    if verbose: print(f's2: sudo ifconfig {s2_iface1_name} hw ether {s2_macs[1]}')
    stdout, stderr = s2.execute(f'sudo ifconfig {s2_iface1_name} hw ether {s2_macs[1]}', quiet=True)
    if verbose: print(f's3: sudo ifconfig {s3_iface0_name} hw ether {s3_macs[0]}')
    stdout, stderr = s3.execute(f'sudo ifconfig {s3_iface0_name} hw ether {s3_macs[0]}', quiet=True)
    if verbose: print(f's3: sudo ifconfig {s3_iface1_name} hw ether {s3_macs[1]}')
    stdout, stderr = s3.execute(f'sudo ifconfig {s3_iface1_name} hw ether {s3_macs[1]}', quiet=True)
    if verbose: print(f's4: sudo ifconfig {s4_iface0_name} hw ether {s4_macs[0]}')
    stdout, stderr = s4.execute(f'sudo ifconfig {s4_iface0_name} hw ether {s4_macs[0]}', quiet=True)
    if verbose: print(f's4: sudo ifconfig {s4_iface1_name} hw ether {s4_macs[1]}')
    stdout, stderr = s4.execute(f'sudo ifconfig {s4_iface1_name} hw ether {s4_macs[1]}', quiet=True)
    if verbose: print(f's4: sudo ifconfig {s4_iface2_name} hw ether {s4_macs[2]}')
    stdout, stderr = s4.execute(f'sudo ifconfig {s4_iface2_name} hw ether {s4_macs[2]}', quiet=True)
    if verbose: print(f'h2: sudo ifconfig {h2_iface_name} hw ether {h2_mac}')
    stdout, stderr = h2.execute(f'sudo ifconfig {h2_iface_name} hw ether {h2_mac}', quiet=True)
    
    # IPs
    if verbose: print(f's1: sudo ip addr add {s1_ipv4}/24 dev {s1_iface0_name}')
    stdout, stderr = s1.execute(f'sudo ip addr add {s1_ipv4}/24 dev {s1_iface0_name}', quiet=True)
    if verbose: print(f'h1: sudo ip addr add {h1_ipv4}/24 dev {h1_iface_name}')
    stdout, stderr = h1.execute(f'sudo ip addr add {h1_ipv4}/24 dev {h1_iface_name}', quiet=True)
    if verbose: print(f's2: sudo ip addr add {s2_ipv4}/24 dev {s2_iface0_name}')
    stdout, stderr = s2.execute(f'sudo ip addr add {s2_ipv4}/24 dev {s2_iface0_name}', quiet=True)
    if verbose: print(f's3: sudo ip addr add {s3_ipv4}/24 dev {s3_iface0_name}')
    stdout, stderr = s3.execute(f'sudo ip addr add {s3_ipv4}/24 dev {s3_iface0_name}', quiet=True)
    if verbose: print(f's4: sudo ip addr add {s4_ipv4}/24 dev {s4_iface0_name}')
    stdout, stderr = s4.execute(f'sudo ip addr add {s4_ipv4}/24 dev {s4_iface0_name}', quiet=True)
    if verbose: print(f'h2: sudo ip addr add {h2_ipv4}/24 dev {h2_iface_name}')
    stdout, stderr = h2.execute(f'sudo ip addr add {h2_ipv4}/24 dev {h2_iface_name}', quiet=True)

    print("✅ DONE! Configuring MAC and IP Addresses")
except Exception as e:
    print(e)

##  Configure forwarding and routing

In [ ]:
ips = {}
ips["h1"] = h1_ipv4
ips["h2"] = h2_ipv4
ifaces = {}
ifaces["h1"] = h1_iface_name
ifaces["h2"] = h2_iface_name
nhop_macs = {}
nhop_macs["h1"] = s1_macs[1]
nhop_macs["h2"] = s4_macs[2]
hosts = [h1, h2]

for host in hosts:
    hostname = host.get_name()
    if verbose: print(f'{hostname}:sudo ifconfig {ifaces[hostname]} del {ips[hostname]} && sudo ip route del {subnet}')
    stdout, stderr = host.execute(f'sudo ifconfig {ifaces[hostname]} del {ips[hostname]} && sudo ip route del {subnet}', quiet=True)
    if verbose: print(f'{hostname}:sudo ifconfig {ifaces[hostname]} add {ips[hostname]} up && sudo ip route add {subnet} dev {ifaces[hostname]}')
    stdout, stderr = host.execute(f'sudo ifconfig {ifaces[hostname]} add {ips[hostname]} up && sudo ip route add {subnet} dev {ifaces[hostname]}', quiet=True)
    
    for dest in hosts:
        destname = dest.get_name()
        if dest == host:
            continue
        if verbose: print(f'{hostname}:sudo ip neigh del {ips[destname]} dev {ifaces[hostname]}')
        stdout, stderr = host.execute(f'sudo ip neigh del {ips[destname]} dev {ifaces[hostname]}', quiet=True)
        if verbose: print(f'{hostname}:sudo ip neigh add {ips[destname]} lladdr {nhop_macs[destname]} dev {ifaces[hostname]}')
        stdout, stderr = host.execute(f'sudo ip neigh add {ips[destname]} lladdr {nhop_macs[destname]} dev {ifaces[hostname]}', quiet=True)
        
print("✅ DONE! Configuring network interfaces, IP addresses, routes, and neighbor entries")

## Configure ARP

Ensure that ARP packets are not sent towards the switch, MAC addresses are hardcoded on the hosts.

In [ ]:
h1.execute(f"sudo arp -s {h2_ipv4} {s1_macs[1]}")
h2.execute(f"sudo arp -s {h1_ipv4} {s4_macs[2]}")
s1.execute(f"sudo arp -s {h1_ipv4} {h1_mac}")
s1.execute(f"sudo arp -s {h2_ipv4} {s1_macs[2]}")
s2.execute(f"sudo arp -s {h2_ipv4} {s4_macs[0]}")
s2.execute(f"sudo arp -s {h1_ipv4} {s1_macs[0]}")
s3.execute(f"sudo arp -s {h2_ipv4} {s4_macs[1]}")
s3.execute(f"sudo arp -s {h1_ipv4} {s1_macs[2]}")
s4.execute(f"sudo arp -s {h1_ipv4} {s3_macs[1]}")
s4.execute(f"sudo arp -s {h2_ipv4} {h2_mac}")

print("✅ DONE! Adding static ARP entries")

## MAC address configurations

In [ ]:
for node in [h1,h2,s1,s2,s3,s4]:
    print(f"\nMAC address configuration for {node.get_name()}") 
    node.execute('arp -a')

## Open required ports

In [ ]:
print("Configuring h2... to open the required ports!")
h2.execute(f"echo '{h2_ipv4} {h2.get_name()}' | sudo tee -a /etc/hosts")
# h2.execute_thread("sudo killall nc", quiet=True)
h2.execute_thread("sudo nc -u -l 53 >/dev/null 2>&1")
h2.execute_thread("sudo nc -l -w 1 80 >/dev/null 2>&1")
h2.execute("sudo iptables -I INPUT -p udp --dport 53 -j DROP")
h2.execute("sudo iptables -I INPUT -p tcp --dport 80 -j DROP")
print("✅ DONE!")

---
## Reinitialize Variables 🔁

If you ever close the Jupyter kernel, or it crashes, all variable values are lost. The below cell can be executed to reset all the variable values.
Note that if the slice expires (24 hours after creation) you must start from the beginning. Any progress made in files (the shell script or P4 code) is safe as long as it is saved.

> Make sure to place your student ID in place of "student_id".

In [ ]:
## Reset

student_id = ""
verbose = 0
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network
config_name = f"{student_id}_assignment5.txt"
config_file = open(config_name, 'r')
configs = [line.rstrip('\n') for line in config_file]

#Config values
subnet = IPv4Network(configs[0])
h1_ipv4 = IPv4Address(configs[1])
h2_ipv4 = IPv4Address(configs[2])
s1_macs = configs[3].split(',')
s2_macs = configs[4].split(',')
s3_macs = configs[5].split(',')
s4_macs = configs[6].split(',')
h1_mac = configs[7]
h2_mac = configs[8]
s1_ipv4 = IPv4Address(configs[9])
s2_ipv4 = IPv4Address(configs[10])
s3_ipv4 = IPv4Address(configs[11])
s4_ipv4 = IPv4Address(configs[12])

#Slice values
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()
slice_name = student_id+"_assignment5"
slice = fablib.get_slice(name=slice_name)
s1 = slice.get_node(name='s1')
s2 = slice.get_node(name='s2')
s3 = slice.get_node(name='s3')
s4 = slice.get_node(name='s4')
h1 = slice.get_node(name='h1')
h2 = slice.get_node(name='h2')
s1_iface0_name = s1.get_interface(network_name='s1s2').get_device_name()
s1_iface1_name = s1.get_interface(network_name='s1h1').get_device_name()
s1_iface2_name = s1.get_interface(network_name='s1s3').get_device_name()
h1_iface_name = h1.get_interface(network_name='s1h1').get_device_name()
s2_iface0_name = s2.get_interface(network_name='s1s2').get_device_name()
s2_iface1_name = s2.get_interface(network_name='s2s4').get_device_name()
s3_iface0_name = s3.get_interface(network_name='s1s3').get_device_name()
s3_iface1_name = s3.get_interface(network_name='s3s4').get_device_name()
s4_iface0_name = s4.get_interface(network_name='s2s4').get_device_name()
s4_iface1_name = s4.get_interface(network_name='s3s4').get_device_name()
s4_iface2_name = s4.get_interface(network_name='s4h2').get_device_name()
h2_iface_name = h2.get_interface(network_name='s4h2').get_device_name()
switches = [s1, s2, s3, s4]
hosts = [h1, h2]
ips = {}
ips["h1"] = h1_ipv4
ips["h2"] = h2_ipv4
ifaces = {}
ifaces["h1"] = h1_iface_name
ifaces["h2"] = h2_iface_name
nhop_macs = {}
nhop_macs["h1"] = s1_macs[1]
nhop_macs["h2"] = s4_macs[2]

print("✅ DONE!")

# Part A: Tunnelling using a Custom Tunnel Header

In this part of the assignment, you will implement a custom tunneling protocol using a custom tunnel header. The goal is to encapsulate packets with the custom header at the ingress switch (S1), forward them through the transit switches (S2 and S3) based on the custom header fields, and decapsulate them at the egress switch (S4) before sending them to the destination host (h2).

The packet's protocol stack should be as follows:

- **Original Stack -** `Ethernet` | `IPv4` | `TCP/UDP`
- **After Encapsulation -** `Ethernet` | `CustomHeader` | `IPv4` | `TCP/UDP`
- **After Decapsulation -** `Ethernet` | `IPv4` | `TCP/UDP`

### Switch S1 (Ingress Switch):

Your task is to modify the provided `s1.p4` code to implement the functionality of the ingress switch (S1) for the custom tunneling protocol. You need to modify the ingress processing to add the custom tunnel header to the packet, and update the deparser to emit the custom tunnel header.

The custom tunnel header should include the following fields:

- `proto_id`: Protocol identifier for the custom tunnel header. It should be assigned the value of the next protocol's number in the stack (will be used during de-parsing).

- `dst_id`: Destination ID for the tunneled packet. All packets must be forwarded based on this field once the packet is tunnelled (encapsulated). Forward the encapsulated packets to either switch S2 or S3 based on the dst_id field.

> Note: In the next part of the assignment, you will get to extend the functionality to include ECMP load balancing. For now, focus on forwarding packets on one of the switches only.


### Switches S2 and S3 (Transit Switches):

For `s2.p4` and `s3.p4`, you need to implement the functionality of the transit switches in the custom tunneling protocol. Update the parser to extract the custom tunnel header fields, modify the ingress processing to forward the packet based on the custom header fields, and update the deparser to emit the custom tunnel header.

> Note: The code for `s2.p4` and `s3.p4` will be similar, with the only difference being the specific table entries and output port values you define in the control plane based on your topology.

### Switch S4 (Egress Switch):

For `s4.p4`, you need to implement the functionality of the egress switch in the custom tunneling protocol. Update the parser to extract the custom tunnel header fields, modify the ingress processing to forward the packet based on the custom header fields, implement the egress processing to remove the custom tunnel header, and update the deparser accordingly.

### Packet Flow

Here's a general overview of how packets should flow from host h1 to host h2 using the custom tunneling protocol:

1. Host h1 sends a packet to switch S1.
2. S1 encapsulates the packet with the custom tunnel header and forwards it to S2 or S3.
3. S2 or S3 forwards the encapsulated packet based on the custom header fields.
4. S4 receives the encapsulated packet, removes the custom tunnel header, and forwards the original packet to host h2.

Your task is to implement the necessary logic in the P4 programs (`s1.p4`, `s2.p4`, `s3.p4`, and `s4.p4`) to achieve this packet flow and implement the custom tunneling protocol.

Feel free to explore different approaches and test your implementation thoroughly to ensure that packets are correctly encapsulated, forwarded, and decapsulated at each switch.

## Switch and Host Address Mapping

In [ ]:
print(f"\ns1's MAC addresses are (in order from port 0 to port 2):\n{s1_macs[0]} ({s1_iface0_name})\n{s1_macs[1]} ({s1_iface1_name})\n{s1_macs[2]} ({s1_iface2_name})")
print(f"s1's IPv4 address is {s1_ipv4}")
print(f"s1's IPv4 address on {s1_iface0_name} is {s1_ipv4}")
print(f"\ns2's MAC addresses are (in order from port 0 to port 1):\n{s2_macs[0]} ({s2_iface0_name})\n{s2_macs[1]} ({s2_iface1_name})")
print(f"s2's IPv4 address on {s2_iface0_name} is {s2_ipv4}")
print(f"\ns3's MAC addresses are (in order from port 0 to port 1):\n{s3_macs[0]} ({s3_iface0_name})\n{s3_macs[1]} ({s3_iface1_name})")
print(f"s3's IPv4 address on {s3_iface0_name} is {s3_ipv4}")
print(f"\ns4's MAC addresses are (in order from port 0 to port 2):\n{s4_macs[0]} ({s4_iface0_name})\n{s4_macs[1]} ({s4_iface1_name})\n{s4_macs[2]} ({s4_iface2_name})")
print(f"s4's IPv4 address on {s4_iface0_name} is {s4_ipv4}")
print("\n")
print("For the above:")
print(f"- s1 is connected to h1 on port 1 ({s1_iface1_name}), to s2 on port 0 ({s1_iface0_name}), and to s3 on port 2 ({s1_iface2_name}).")
print(f"- s2 is connected to s1 on port 0 ({s2_iface0_name}) and to s4 on port 1 ({s2_iface1_name}).")
print(f"- s3 is connected to s1 on port 0 ({s3_iface0_name}) and to s4 on port 1 ({s3_iface1_name}).")
print(f"- s4 is connected to s2 on port 0 ({s4_iface0_name}), to s3 on port 1 ({s4_iface1_name}), and to h2 on port 2 ({s4_iface2_name}).")
print("\n")
print(f"h1's MAC address is {h1_mac} and its IPv4 address is {h1_ipv4}")
print(f"h1 is connected to s1 on interface {h1_iface_name}")
print(f"\nh2's MAC address is {h2_mac} and its IPv4 address is {h2_ipv4}")
print(f"h2 is connected to s4 on interface {h2_iface_name}")

### Helper to Copy SSH Url to Clipboard

In [ ]:
import ipywidgets as widgets
from IPython.display import display, Javascript
import time

def display_ssh_copy_buttons(slice_name, button_style="primary"):
    try:
        slice = fablib.get_slice(name=slice_name)
        nodes = slice.get_nodes()
        
        def create_ssh_copy_button(ssh_command,node_name):
            ssh_copy_btn = widgets.Button(description=f'Copy SSH for {node_name}', button_style=button_style, icon='clipboard', style={"text_color":"white"})
            ssh_copy_output = widgets.Output()
            
            def ssh_copy_btn_event_handler(b):
                with ssh_copy_output:
                    display(Javascript(f'''
                        navigator.clipboard.writeText("{ssh_command}");
                    '''))
                    ssh_copy_btn.description = 'Copied!'
                    ssh_copy_btn.disabled = True
                    time.sleep(1) 
                    ssh_copy_btn.description = f'Copy SSH for {node_name}'
                    ssh_copy_btn.disabled = False
            
            ssh_copy_btn.on_click(ssh_copy_btn_event_handler)
            return ssh_copy_btn, ssh_copy_output
        
        ssh_widgets_list = []
        for node in nodes:
            node_name = node.get_name()
            ssh_command = node.get_ssh_command()
            ssh_copy_btn, ssh_copy_output = create_ssh_copy_button(ssh_command, node_name)
            ssh_widgets_list.append(widgets.VBox([ssh_copy_btn, ssh_copy_output]))
        
        ssh_table = widgets.HBox(ssh_widgets_list)
        display(ssh_table)
    except Exception as e:
        print(f"Exception: {e}")

display_ssh_copy_buttons(slice_name)

## Create BMv2 helper scripts - this cell only needs to run once

In [ ]:
# Create BMv2 helper scripts - this cell only needs to run once
s_ifaces = {}
s_ifaces["s1"] = [s1_iface0_name, s1_iface1_name, s1_iface2_name]
s_ifaces["s2"] = [s2_iface0_name, s2_iface1_name]
s_ifaces["s3"] = [s3_iface0_name, s3_iface1_name]
s_ifaces["s4"] = [s4_iface0_name, s4_iface1_name, s4_iface2_name]

for switch in [s1, s2, s3, s4]:
    switchname = switch.get_name()
    outfile = "scripts/BMv2/run_bmv2_" + switchname + ".sh"
    with open(outfile, 'w') as out:
        out.write("#!/bin/bash\n\n")
        out.write(f"p4c --target bmv2 --arch v1model --std p4-16 ~/{switchname}.p4 -o ~\n\n")
        
        if switchname == "s1":
            out.write(f"sudo simple_switch -i 0@{s_ifaces[switchname][0]} -i 1@{s_ifaces[switchname][1]} -i 2@{s_ifaces[switchname][2]} ~/{switchname}.json --log-console")
        elif switchname == "s2" or switchname == "s3":
            out.write(f"sudo simple_switch -i 0@{s_ifaces[switchname][0]} -i 1@{s_ifaces[switchname][1]} ~/{switchname}.json --log-console")
        elif switchname == "s4":
            out.write(f"sudo simple_switch -i 0@{s_ifaces[switchname][0]} -i 1@{s_ifaces[switchname][1]} -i 2@{s_ifaces[switchname][2]} ~/{switchname}.json --log-console")
        out.close()

In [ ]:
# Build mass upload file 
try:
    scps = []
    outfile = "scripts/uploader.sh"
    for switch in [s1, s2, s3, s4]:
        switchname = switch.get_name()
        ssh = slice.get_node(name=switchname).get_ssh_command()
        param = ssh.split("ssh ")[1]
        scp = f"scp {param}"
        scp_addr = f'[{scp.split("@")[1]}]'
        scp_config = scp.split('@')[0]
        scp_base = f"{scp_config}@{scp_addr}"
        scp_bmv2 = f"{scp_base.split('ubuntu')[0]}scripts/BMv2/run_bmv2_{switchname}.sh ubuntu{scp_base.split('ubuntu')[1]}:~"
        scp_rules = f"{scp_base.split('ubuntu')[0]}rules/add_rules_{switchname}.sh ubuntu{scp_base.split('ubuntu')[1]}:~"
        scp_p4 = f"{scp_base.split('ubuntu')[0]}p4/{switchname}.p4 ubuntu{scp_base.split('ubuntu')[1]}:~"
        scps.append(scp_bmv2)
        scps.append(scp_rules)
        scps.append(scp_p4)

    with open(outfile, 'w') as out:
        out.write("#!/bin/bash\n\n")
        for command in scps:
            out.write(f"{command}\n")
except Exception as e:
    print(f"Exception: {e}")


In [ ]:
 %%sh
# Uploader - initial upload, only needed once
chmod u+x scripts/uploader.sh
./scripts/uploader.sh

Ensure the `rules/add_rules_s[1-5].sh` are populated before running the below cell

### <-------------------------Re-Run Cells From Here On Code Change-------------------------------------->

If you modify your `s1.p4`, `s2.p4`, `s3.p4`, `s4.p4`, `add_rules_s1.sh`, `add_rules_s2.sh`, `add_rules_s3.sh`, `add_rules_s4.sh`, then re-run the below cells up until `#SSH-Keys` section

In [ ]:
outfile = "scripts/uploader2.sh"
with open(outfile, 'w') as out:
    out.write("#!/bin/bash\n\n")
    for command in scps[:12]:
        out.write(command + '\n')
stdout, stderr = s1.execute("chmod u+x run_bmv2_s1.sh && chmod u+x add_rules_s1.sh", quiet=True)
stdout, stderr = s2.execute("chmod u+x run_bmv2_s2.sh && chmod u+x add_rules_s2.sh", quiet=True)
stdout, stderr = s3.execute("chmod u+x run_bmv2_s3.sh && chmod u+x add_rules_s3.sh", quiet=True)
stdout, stderr = s4.execute("chmod u+x run_bmv2_s4.sh && chmod u+x add_rules_s4.sh", quiet=True)

In [ ]:
 %%sh
# Uploader2 - pushes changes for switch files, execute as needed
chmod u+x scripts/uploader2.sh
./scripts/uploader2.sh

# Custom Header Tunnel Forward Test

Kill any previously running server and start the switches. Capture logs into a file called `logs` in the background. Add rules once the switches are up.Kill any previously running server and start the switches. Capture logs into a file called `logs` in the background. Add rules once the switches are up.

In [ ]:
# Custom Header Tunnel Forward Test
from time import sleep

verbose = 1

nodes = [s1,s2,s3,s4,h2,h1]

# Kill any running switch
for node in nodes[:4]:
    stdout, stderr = node.execute("sudo killall simple_switch", quiet=True)

# Run the switch and save logs
s1.execute_thread("./run_bmv2_s1.sh > logs 2>&1")
s2.execute_thread("./run_bmv2_s2.sh > logs 2>&1")
s3.execute_thread("./run_bmv2_s3.sh > logs 2>&1")
s4.execute_thread("./run_bmv2_s4.sh > logs 2>&1")

sleep(2)

# Add Control Plane Table Rules
s1.execute("./add_rules_s1.sh", quiet=True)
s2.execute("./add_rules_s2.sh", quiet=True)
s3.execute("./add_rules_s3.sh", quiet=True)
s4.execute("./add_rules_s4.sh", quiet=True)

print("✅Done! Switches are up and running!")

## Get SSH connections to Debug

In [ ]:
display_ssh_copy_buttons(slice_name)

## Run Simulation

In [ ]:
# Update the interfaces based on your topology
s1.execute_thread(f"sudo tcpdump -i {s1_iface1_name} -Q in -w s1_capture.pcap")
s2.execute_thread(f"sudo tcpdump -i {s2_iface0_name} -Q in -w s2_capture.pcap")
s3.execute_thread(f"sudo tcpdump -i {s3_iface0_name} -Q in -w s3_capture.pcap")
s4.execute_thread(f"sudo tcpdump -i {s4_iface0_name} -Q in -w s4_iface0_capture.pcap")
s4.execute_thread(f"sudo tcpdump -i {s4_iface1_name} -Q in -w s4_iface1_capture.pcap")
h2.execute_thread(f"sudo tcpdump -i {h2_iface_name} -Q in -w h2_receive_test.pcap")

sleep(1)

commands = [
    f"sudo python3 -c 'from scapy.all import *; sendp(Ether(src=\"{h1_mac}\",dst=\"{s1_macs[1]}\")/IP(src=\"{h1_ipv4}\",dst=\"{h2_ipv4}\")/UDP(sport=42567, dport=53), iface=\"{h1_iface_name}\")'",
    f"sudo python3 -c 'from scapy.all import *; sendp(Ether(src=\"{h1_mac}\",dst=\"{s1_macs[1]}\")/IP(src=\"{h1_ipv4}\",dst=\"{h2_ipv4}\")/TCP(sport=35678, dport=80), iface=\"{h1_iface_name}\")'"
]

# Send the TCP and UDP packet from h1 to h2
original_packets = []
for command in commands:
    print(command)
    stdout, stderr = h1.execute(command)
    original_packets.append(stdout)

# Wait for a short duration to allow packet processing and forwarding
sleep(2)

# Stop the packet captures on switches
for node in nodes[:5]:
    node.execute("sudo killall tcpdump")

# Transfer the PCAP files from switches to a central location (adjust paths as needed)
print("Analyzing captured packets using tshark...")

# Define the tcpdump options
flags = ""

# Analyze packets captured on s1
print("\nPackets captured on s1:")
s1.execute(f"tshark -r s1_capture.pcap")
if verbose:
    print("Size of each packet on ingress to s1")
    s1.execute(f"tshark -r s1_capture.pcap -T fields -e frame.len -E separator=,")

# Analyze packets captured on s2
print("\nPackets captured on s2:")
s2.execute(f"tshark -r s2_capture.pcap {flags}")
if verbose:
    print("\nSize of each packet on ingress to s2")
    s2.execute(f"tshark -r s2_capture.pcap -T fields -e frame.len -E separator=,")

# Analyze packets captured on s3
print("\nPackets captured on s3:")
s3.execute(f"tshark -r s3_capture.pcap {flags}")
if verbose:
    print("\nSize of each packet on ingress to s3")
    s3.execute(f"tshark -r s3_capture.pcap -T fields -e frame.len -E separator=,")

# Analyze packets captured on s4
# Analyze packets captured on s4 interface 1
print("\nPackets captured on s4 interface 0:")
s4.execute(f"tshark -r s4_iface0_capture.pcap {flags}")
if verbose:
    print("\nSize of each packet on ingress to s4 interface 0")
    s4.execute(f"tshark -r s4_iface0_capture.pcap -T fields -e frame.len -E separator=,")

# Analyze packets captured on s4 interface 2
print("\nPackets captured on s4 interface 1:")
s4.execute(f"tshark -r s4_iface1_capture.pcap {flags}")
if verbose:
    print("\nSize of each packet on ingress to s4 interface 1")
    s4.execute(f"tshark -r s4_iface1_capture.pcap -T fields -e frame.len -E separator=,")


# Display the captured packet on h2
print("\nPacket captured on h2:")
h2.execute(f"tshark -r h2_receive_test.pcap")
if verbose:
    print("\nSize of each packet on ingress to h2")
    h2.execute(f"tshark -r h2_receive_test.pcap -T fields -e frame.len -E separator=,")


## Auto-Grader - Part A

🚫 Please do NOT edit this cell! Make sure to test your score once you've run the simulation to verify the correctness of your code. These test cases are specific to Part A. If the Part B simulation was run then Part A test cases fail wil fail on re-rerunning of the below cell. So make sure to throughly test Part A before moving on to Part B!

In [ ]:
total_points = 8
points_earned = 0
verbose = 1  # Set to 1 to display detailed test conditions

def getResult(condition=False, points=2):
    global points_earned
    if condition:
        points_earned += points
        return '✅ PASS |'
    else:
        return '❌ FAIL |'

# Test 1: IPv4 Packets (TCP and UDP) captured on S1 (2 points)
stdout_tcp, stderr_tcp = s1.execute(f"tshark -r s1_capture.pcap -Y 'ip.src=={h1_ipv4} and ip.dst=={h2_ipv4} and tcp.port==35678' -T fields -e frame.number -e frame.len",quiet=True)
stdout_udp, stderr_udp = s1.execute(f"tshark -r s1_capture.pcap -Y 'ip.src=={h1_ipv4} and ip.dst=={h2_ipv4} and udp.port==42567' -T fields -e frame.number -e frame.len",quiet=True)
tcp_packets = stdout_tcp.strip().split('\n')
udp_packets = stdout_udp.strip().split('\n')
condition = len(tcp_packets) == 1 and len(udp_packets) == 1 and tcp_packets[0].split()[1] == '56' and udp_packets[0].split()[1] == '56'
result = getResult(condition)
print(f"\n{result} Test 1: IPv4 Packets (1 TCP and 1 UDP) captured on S1 with length 60 bytes each (2 points)")
if verbose:
    print(f"    \tTCP packets captured on S1: {len(tcp_packets)}")
    print(f"    \tUDP packets captured on S1: {len(udp_packets)}")
    print(f"    \tTCP packet length: {tcp_packets[0].split()[1] if tcp_packets else 'N/A'}")
    print(f"    \tUDP packet length: {udp_packets[0].split()[1] if udp_packets else 'N/A'}")
assert condition, "Test 1 failed: Expected 1 TCP and 1 UDP packet with length 60 bytes each captured on S1"

# Test 2: Encapsulated IPv4 Packet with the custom tunnel header captured on S2 or S3 (2 points)
s2_stdout, s2_stderr = s2.execute(f"tshark -r s2_capture.pcap -Y 'eth.type==0x1212' -T fields -e frame.number -e frame.len",quiet=True)
s3_stdout, s3_stderr = s3.execute(f"tshark -r s3_capture.pcap -Y 'eth.type==0x1212' -T fields -e frame.number -e frame.len",quiet=True)
s2_packets = s2_stdout.strip().split('\n')
s3_packets = s3_stdout.strip().split('\n')
condition = (s2_packets and all(packet.split()[1] == '60' for packet in s2_packets)) or (s3_packets and all(packet.split()[1] == '60' for packet in s3_packets))
result = getResult(condition)
print(f"\n{result} Test 2: Encapsulated IPv4 Packet with the custom tunnel header (EtherType 0x1212) captured on S2 or S3 with length 60 bytes (2 points)")
if verbose:
    print(f"    \tEncapsulated IPv4 packets captured on S2: {len(s2_packets)}")
    print(f"    \tEncapsulated IPv4 packets captured on S3: {len(s3_packets)}")
    print(f"    \tPacket lengths on S2: {', '.join(packet.split()[1] for packet in s2_packets) if s2_packets else 'N/A'}")
    print(f"    \tPacket lengths on S3: {', '.join(packet.split()[1] for packet in s3_packets) if s3_packets else 'N/A'}")
assert condition, "Test 2 failed: No encapsulated IPv4 packets with the custom tunnel header (EtherType 0x1212) and length 60 bytes captured on S2 or S3"

# Test 3: 2x Encapsulated IPv4 Packets with the custom tunnel header captured on S4 (4 points)
stdout1, stderr1 = s4.execute(f"tshark -r s4_iface0_capture.pcap -Y 'eth.type==0x1212' -T fields -e frame.number -e frame.len",quiet=True)
stdout2, stderr2 = s4.execute(f"tshark -r s4_iface1_capture.pcap -Y 'eth.type==0x1212' -T fields -e frame.number -e frame.len",quiet=True)
s4_iface0_packets = stdout1.strip().split('\n')
s4_iface1_packets = stdout2.strip().split('\n')
condition = len(s4_iface0_packets) + len(s4_iface1_packets) == 2 and all(packet.split()[1] == '60' for packet in s4_iface0_packets + s4_iface1_packets)
result = getResult(condition)
points = 4 if condition else 0
print(f"\n{result} Test 3: 2x Encapsulated IPv4 Packets with the custom tunnel header (EtherType 0x1212) captured on S4 with length 60 bytes each ({points} points)")
if verbose:
    print(f"    \tEncapsulated IPv4 packets captured on S4 (iface0): {len(s4_iface0_packets)}")
    print(f"    \tEncapsulated IPv4 packets captured on S4 (iface1): {len(s4_iface1_packets)}")
    print(f"    \tTotal encapsulated IPv4 packets captured on S4: {len(s4_iface0_packets) + len(s4_iface1_packets)}")
    print(f"    \tPacket lengths on S4: {', '.join(packet.split()[1] for packet in s4_iface0_packets + s4_iface1_packets)}")
assert condition, "Test 3 failed: Expected 2 encapsulated IPv4 packets with the custom tunnel header (EtherType 0x1212) and length 60 bytes each captured on S4"

# Test 4: Decapsulated (original) IPv4 (TCP + UDP) Packets captured on h2 (2 points)
stdout_tcp, stderr_tcp = h2.execute(f"tshark -r h2_receive_test.pcap -Y 'ip.src=={h1_ipv4} and ip.dst=={h2_ipv4} and tcp.port==35678' -T fields -e frame.number -e frame.len",quiet=True)
stdout_udp, stderr_udp = h2.execute(f"tshark -r h2_receive_test.pcap -Y 'ip.src=={h1_ipv4} and ip.dst=={h2_ipv4} and udp.port==42567' -T fields -e frame.number -e frame.len",quiet=True)
h2_tcp_packets = stdout_tcp.strip().split('\n')
h2_udp_packets = stdout_udp.strip().split('\n')
condition=len(h2_tcp_packets) == 1 and len(h2_udp_packets) == 1 and h2_tcp_packets[0].split()[1] == '56' and h2_udp_packets[0].split()[1] == '56'
result = getResult(condition)
print(f"\n{result} Test 4: Decapsulated (original) IPv4 Packets (1 TCP and 1 UDP) captured on h2 with length 60 bytes each (2 points)")
if verbose:
    print(f"    \tDecapsulated TCP packets captured on h2: {len(h2_tcp_packets)}")
    print(f"    \tDecapsulated UDP packets captured on h2: {len(h2_udp_packets)}")
    print(f"    \tTCP packet length: {h2_tcp_packets[0].split()[1] if h2_tcp_packets else 'N/A'}")
    print(f"    \tUDP packet length: {h2_udp_packets[0].split()[1] if h2_udp_packets else 'N/A'}")
assert condition,"Test 4 failed: Expected 1 TCP and 1 UDP decapsulated IPv4 packet with length 60 bytes each captured on h2"

# Print the total points earned
print(f"\n➡️ Total points earned: {points_earned}/{total_points}")

---
### Verification

To verify the custom tunneling protocol implementation, analyze the captured packets at each switch and the end hosts:

1. h1: Check if the sent packets have the correct source and destination addresses and ports.

2. s1: Verify that the custom tunnel header is added with the correct `proto_id` and `dst_id` fields, and the outer headers are updated. Also, ensure that the ECMP hash is calculated correctly based on the 5-tuple values, and the packets are forwarded to the appropriate output ports (s2 or s3) based on the hash value.

3. s2 and s3: Ensure that the packets with the custom tunnel header are forwarded based on the `dst_id` field, and the outer headers are updated.

4. s4: Check if the packets with the custom tunnel header are received on both input interfaces (from s2 and s3). Verify that the custom tunnel header is removed for packets destined for h2, and the original packet is extracted correctly.

5. h2: Confirm that the decapsulated packet is received and matches the original packet sent from h1.

### Grading Criteria

The total points possible for Part A is 10 points, distributed as follows:

1. IPv4 Packets (TCP and UDP) captured on S1 (2 points)
   - Test for the presence of IPv4 packets with the correct source and destination addresses and ports.

2. Encapsulated IPv4 Packet with the custom tunnel header captured on S2 or S3 or both (2 points)
   - Test for the presence of the custom tunnel header with the correct `proto_id` and `dst_id` fields.

3. 2x Encapsulated IPv4 Packets with the custom tunnel header captured on S4 (2 points)
   - Verify that two encapsulated packets with the custom tunnel header are received on both input interfaces of S4.

4. Decapsulated (original) IPv4 Packets captured on h2 (2 points)
   - Test for the presence of the decapsulated IPv4 packets (TCP and UDP) that match the original packets sent from h1.

---
### 🚨 Attention 🚨

It is crucial to ensure that **Part A (Custom Header Encapsulation + Forwarding Implementation)** is thoroughly tested and working correctly before moving on to **Part B (Load Balancer Implementation)**. Proceeding with Part B without a fully functional and verified Part A can lead to compounded issues and difficulties in debugging.

---

## Part B: ECMP Load Balancer Implementation

In this part, you will extend the functionality of the P4 program to implement an Equal-Cost Multi-Path (ECMP) load balancer on switch S1 using 5-tuple hashing. The load balancer will distribute the traffic from host h1 to host h2 between switches S2 and S3 based on the hash result.

5-tuple hashing is a technique used in network load balancing and packet forwarding to distribute traffic across multiple paths or servers. It involves calculating a hash value based on five key fields from the packet headers: source IP address, destination IP address, protocol, source port, and destination port. The hash value is typically used to select one of the available paths or servers in a load-balanced system.

The general syntax for calculating the 5-tuple hash in a packet processing pipeline is as follows:

```python
hash(<hash_field>,
     <hash_algorithm>,
     <initial_value>,
     { <tuple_field_1>, <tuple_field_2>, <tuple_field_3>, <tuple_field_4>, <tuple_field_5> },
     <output_width>);
```

- `<hash_field>`: The metadata field where the calculated hash value will be stored.
- `<hash_algorithm>`: The hash algorithm to be used, such as CRC16, CRC32, or a custom algorithm.
- `<initial_value>`: The initial value for the hash calculation.
- `<tuple_field_1>` to `<tuple_field_5>`: The five fields that form the 5-tuple, typically including the source IP address, destination IP address, protocol, source port, and destination port.
- `<output_width>`: The width of the resulting hash value in bits.

> Note: By selecting the right hash algorithm and the fields used in the 5-tuple, you can achieve an even distribution of traffic across the available paths.## Part B: ECMP Load Balancer Implementation

### Switch S1 Tasks:

- Receive packets from host h1.
- Perform 5-tuple hashing on the packets to determine the next-hop switch (S2 or S3).
- Based on the hash result, select the next-hop switch (S2 or S3) for packet forwarding.
- Forward the packet to the selected switch using the appropriate output port.

### Packet Flow

When a packet arrives at switch S1 from host h1:
1. S1 performs 5-tuple hashing on the packet's header fields (e.g., source IP, destination IP, source port, destination port, and protocol).
2. Based on the hash result, S1 selects the next-hop switch (S2 or S3) for packet forwarding.
3. S1 forwards the packet to the selected switch using the appropriate output port.
4. The receiving switch (S2 or S3) forwards the packet to switch S4.
5. S4 forwards the packet to host h2 based on the destination IP address.

Please note that you'll need to modify the P4 program of switch S1 to implement the necessary packet processing logic and forwarding rules for the ECMP load balancer. The P4 programs for switches S2, S3, and S4 remain unchanged.

Ensure the `rules/add_rules_s[1-5].sh` are populated before running the below cell

### <-------------------------Re-Run Cells From Here On Code Change-------------------------------------->

If you modify your `s1.p4`, `s2.p4`, `s3.p4`, `s4.p4`, `add_rules_s1.sh`, `add_rules_s2.sh`, `add_rules_s3.sh`, `add_rules_s4.sh`, then re-run the below cells up until `#SSH-Keys` section

In [ ]:
 %%sh
# Uploader2 - pushes changes for switch files, execute as needed
chmod u+x scripts/uploader2.sh
./scripts/uploader2.sh

# SSH Keys

In [ ]:
display_ssh_copy_buttons(slice_name)

## ECMP Load Balancing Test

Kill any previously running server and start the switches. Capture logs into a file called `logs` in the background. Add rules once the switches are up.

In [ ]:
# Custom Tunnel Forward Test with ECMP Load Balancing
from time import sleep

verbose = 1

nodes = [s1,s2,s3,s4,h2,h1]

# Kill any running switch
for node in nodes[:4]:
    stdout, stderr = node.execute("sudo killall simple_switch", quiet=True)

# Run the switch and save logs
s1.execute_thread("./run_bmv2_s1.sh > logs 2>&1")
s2.execute_thread("./run_bmv2_s2.sh > logs 2>&1")
s3.execute_thread("./run_bmv2_s3.sh > logs 2>&1")
s4.execute_thread("./run_bmv2_s4.sh > logs 2>&1")

sleep(2)

# Add Control Plane Table Rules
s1.execute("./add_rules_s1.sh", quiet=True)
s2.execute("./add_rules_s2.sh", quiet=True)
s3.execute("./add_rules_s3.sh", quiet=True)
s4.execute("./add_rules_s4.sh", quiet=True)

print("✅Done! Switches are up and running!")

## Run Simulation

In [ ]:
import random

start_time = time.time()

num_packets = 10 # Total 2*num_packets ie, if set to 10 then, 20 packets will be sent - x10 TCP and x10 UDP

# Update the interfaces based on your topology
s1.execute_thread(f"sudo tcpdump -i {s1_iface1_name} -Q in -w s1_capture.pcap")
s2.execute_thread(f"sudo tcpdump -i {s2_iface0_name} -Q in -w s2_capture.pcap")
s3.execute_thread(f"sudo tcpdump -i {s3_iface0_name} -Q in -w s3_capture.pcap")
s4.execute_thread(f"sudo tcpdump -i {s4_iface0_name} -Q in -w s4_iface0_capture.pcap")
s4.execute_thread(f"sudo tcpdump -i {s4_iface1_name} -Q in -w s4_iface1_capture.pcap")
h2.execute_thread(f"sudo tcpdump -i {h2_iface_name} -Q in -w h2_receive_test.pcap")

sleep(1)

# Send multiple packets from h1 to h2
for i in range(num_packets):
    sport = random.randint(1024, 65535)
    h1.execute(f"sudo python3 -c 'from scapy.all import *; sendp(Ether(src=\"{h1_mac}\",dst=\"{s1_macs[1]}\")/IP(src=\"{h1_ipv4}\",dst=\"{h2_ipv4}\")/UDP(sport={sport}, dport=53), iface=\"{h1_iface_name}\")'", quiet=True)
    h1.execute(f"sudo python3 -c 'from scapy.all import *; sendp(Ether(src=\"{h1_mac}\",dst=\"{s1_macs[1]}\")/IP(src=\"{h1_ipv4}\",dst=\"{h2_ipv4}\")/TCP(sport={sport}, dport=80), iface=\"{h1_iface_name}\")'", quiet=True)

# Wait for a short duration to allow packet processing and forwarding
sleep(2)

# Stop the packet captures on switches
for node in nodes[:5]:
    node.execute("sudo killall tcpdump")

# Transfer the PCAP files from switches to a central location (adjust paths as needed)
print("Analyzing captured packets using tshark...")

# Analyze packets captured on each switch and host
switches = [s1, s2, s3]
switch_names = ["s1", "s2", "s3"]
packet_counts = {}

# Define the tcpdump options
flags = '-Y "tcp or udp"' 

for switch, switch_name in zip(switches, switch_names):
    print(f"\nPackets captured on {switch_name}:")
    stdout, stderr = switch.execute(f"tshark -r {switch_name}_capture.pcap", quiet=True)
    packets = stdout.split("\n")
    packet_counts[switch_name] = len(packets) - 1
    print(f"Total packets captured on {switch_name}: {packet_counts[switch_name]}")

# Analyze packets captured on s4 interface 0
print("\nPackets captured on s4 interface 0:")
stdout, stderr = s4.execute(f"tshark -r s4_iface0_capture.pcap", quiet=True)
packets = stdout.split("\n")
packet_counts["s4_iface0"] = len(packets) - 1
print(f"Total packets captured on s4 interface 0: {packet_counts['s4_iface0']}")

# Analyze packets captured on s4 interface 1
print("\nPackets captured on s4 interface 1:")
stdout, stderr = s4.execute(f"tshark -r s4_iface1_capture.pcap", quiet=True)
packets = stdout.split("\n")
packet_counts["s4_iface1"] = len(packets) - 1
print(f"Total packets captured on s4 interface 1: {packet_counts['s4_iface1']}")

# Analyze packets captured on h2
print("\nPackets captured on h2:")
stdout, stderr = h2.execute(f"tshark -r h2_receive_test.pcap {flags}", quiet=True)
packets = stdout.split("\n")
packet_counts["h2"] = len(packets) - 1
print(f"Total packets captured on h2: {packet_counts['h2']}")

# Verify the distribution of packets between S2 and S3
print("\nECMP Load Balancing Distribution:")
total_packets = packet_counts["s1"]
s2_percentage = (packet_counts["s2"] / total_packets) * 100
s3_percentage = (packet_counts["s3"] / total_packets) * 100
print(f"Packets captured on S2: {s2_percentage:.1f}%")
print(f"Packets captured on S3: {s3_percentage:.1f}%")

end_time = time.time()
execution_time = end_time - start_time
print(f"\nTotal execution time: {execution_time:.1f} seconds")

## Auto-Grader - Part B

🚫 Please do NOT edit this cell! Make sure to test your score once you've run the simulation to verify the correctness of your code. These test cases are specific to Part B. If the Part A simulation was run again then Part B test cases fail will fail on re-rerunning of the below cell. So make sure to run Part B simulation before moving on to testing Part B!

In [ ]:
points_from_a = 0
verbose = 1  # Set to 1 to display detailed test conditions

def getResult(condition=False):
    if condition:
        return '✅ PASS |'
    else:
        return '❌ FAIL |'

print("\nTesting cases for Part A (+1 Bonus point)\n")

# Test 1: IPv4 Packets (TCP and UDP) captured on S1
stdout_tcp, stderr_tcp = s1.execute(f"tshark -r s1_capture.pcap -Y 'ip.src=={h1_ipv4} and ip.dst=={h2_ipv4} and tcp.dstport==80' -T fields -e frame.number -e frame.len",quiet=True)
stdout_udp, stderr_udp = s1.execute(f"tshark -r s1_capture.pcap -Y 'ip.src=={h1_ipv4} and ip.dst=={h2_ipv4} and udp.dstport==53' -T fields -e frame.number -e frame.len",quiet=True)
tcp_packets = stdout_tcp.strip().split('\n')
udp_packets = stdout_udp.strip().split('\n')
condition = len(tcp_packets) == num_packets and len(udp_packets) == num_packets and all(packet.split()[1] == '56' for packet in tcp_packets + udp_packets)
result1 = getResult(condition)
print(f"\n{result1} Test 1: {num_packets} TCP and {num_packets} UDP IPv4 packets captured on S1 with length 56 bytes each")
if verbose:
    print(f"    \tTCP packets captured on S1: {len(tcp_packets)}/{num_packets}")
    print(f"    \tUDP packets captured on S1: {len(udp_packets)}/{num_packets}")
    print(f"    \tTCP packet lengths: {', '.join(packet.split()[1] for packet in tcp_packets)}")
    print(f"    \tUDP packet lengths: {', '.join(packet.split()[1] for packet in udp_packets)}")
assert condition, f"Test 1 failed: Expected {num_packets} TCP and {num_packets} UDP packets with length 56 bytes each captured on S1"

# Test 2: Encapsulated IPv4 Packet with the custom tunnel header captured on S2 or S3
s2_stdout, s2_stderr = s2.execute(f"tshark -r s2_capture.pcap -Y 'eth.type==0x1212' -T fields -e frame.number -e frame.len",quiet=True)
s3_stdout, s3_stderr = s3.execute(f"tshark -r s3_capture.pcap -Y 'eth.type==0x1212' -T fields -e frame.number -e frame.len",quiet=True)
s2_packets = s2_stdout.strip().split('\n')
s3_packets = s3_stdout.strip().split('\n')
condition = len(s2_packets) + len(s3_packets) == 2*num_packets and all(packet.split()[1] == '60' for packet in s2_packets + s3_packets)
result2 = getResult(condition)
print(f"\n{result2} Test 2: {2*num_packets} Encapsulated IPv4 Packets with the custom tunnel header (EtherType 0x1212) captured on S2 and S3 with length 60 bytes each")
if verbose:
    print(f"    \tEncapsulated IPv4 packets captured on S2: {len(s2_packets)}/{num_packets}")
    print(f"    \tEncapsulated IPv4 packets captured on S3: {len(s3_packets)}/{num_packets}")
    print(f"    \tPacket lengths on S2: {', '.join(packet.split()[1] for packet in s2_packets)}")
    print(f"    \tPacket lengths on S3: {', '.join(packet.split()[1] for packet in s3_packets)}")
assert condition, f"Test 2 failed: Expected {2*num_packets} encapsulated IPv4 packets with the custom tunnel header (EtherType 0x1212) and length 60 bytes captured on S2 and S3"

# Test 3: num_packets*x Encapsulated IPv4 Packets with the custom tunnel header captured on S4
stdout1, stderr1 = s4.execute(f"tshark -r s4_iface0_capture.pcap -Y 'eth.type==0x1212' -T fields -e frame.number -e frame.len",quiet=True)
stdout2, stderr2 = s4.execute(f"tshark -r s4_iface1_capture.pcap -Y 'eth.type==0x1212' -T fields -e frame.number -e frame.len",quiet=True)
s4_iface0_packets = stdout1.strip().split('\n')
s4_iface1_packets = stdout2.strip().split('\n')
condition = len(s4_iface0_packets) + len(s4_iface1_packets) == 2*num_packets and all(packet.split()[1] == '60' for packet in s4_iface0_packets + s4_iface1_packets)
result3 = getResult(condition)
print(f"\n{result3} Test 3: {2*num_packets} Encapsulated IPv4 Packets with the custom tunnel header (EtherType 0x1212) captured on S4 with length 60 bytes each")
if verbose:
    print(f"    \tEncapsulated IPv4 packets captured on S4 (iface0): {len(s4_iface0_packets)}/{num_packets}")
    print(f"    \tEncapsulated IPv4 packets captured on S4 (iface1): {len(s4_iface1_packets)}/{num_packets}")
    print(f"    \tPacket lengths on S4 (iface0): {', '.join(packet.split()[1] for packet in s4_iface0_packets)}")
    print(f"    \tPacket lengths on S4 (iface1): {', '.join(packet.split()[1] for packet in s4_iface1_packets)}")
assert condition, f"Test 3 failed: Expected {2*num_packets} encapsulated IPv4 packets with the custom tunnel header (EtherType 0x1212) and length 60 bytes each captured on S4"

# Test 4: Decapsulated (original) IPv4 (TCP + UDP) Packets captured on h2
stdout_tcp, stderr_tcp = h2.execute(f"tshark -r h2_receive_test.pcap -Y 'ip.src=={h1_ipv4} and ip.dst=={h2_ipv4} and tcp.dstport==80' -T fields -e frame.number -e frame.len",quiet=True)
stdout_udp, stderr_udp = h2.execute(f"tshark -r h2_receive_test.pcap -Y 'ip.src=={h1_ipv4} and ip.dst=={h2_ipv4} and udp.dstport==53' -T fields -e frame.number -e frame.len",quiet=True)
h2_tcp_packets = stdout_tcp.strip().split('\n')
h2_udp_packets = stdout_udp.strip().split('\n')
condition = len(h2_tcp_packets) == num_packets and len(h2_udp_packets) == num_packets and all(packet.split()[1] == '56' for packet in h2_tcp_packets + h2_udp_packets)
result4 = getResult(condition)
print(f"\n{result4} Test 4: {num_packets} TCP and {num_packets} UDP Decapsulated (original) IPv4 Packets captured on h2 with length 56 bytes each")
if verbose:
    print(f"   \tDecapsulated TCP packets captured on h2: {len(h2_tcp_packets)}/{num_packets}")
    print(f"   \tDecapsulated UDP packets captured on h2: {len(h2_udp_packets)}/{num_packets}")
    print(f"   \tTCP packet lengths: {', '.join(packet.split()[1] for packet in h2_tcp_packets)}")
    print(f"   \tUDP packet lengths: {', '.join(packet.split()[1] for packet in h2_udp_packets)}")
assert condition, f"Test 4 failed: Expected {num_packets} TCP and {num_packets} UDP decapsulated IPv4 packets with length 56 bytes each captured on h2"

# Assign points for Part A tests
points_from_a = 1 if (result1 == '✅ PASS |' and result2 == '✅ PASS |' and result3 == '✅ PASS |' and result4 == '✅ PASS |') else 0

print(f"\n➡️ Passed all tests from Part A: +1 Bonus Point")

total_points = 4
points_earned = points_from_a

print("\nTesting cases for Part B (ECMP Load Balancing)")

# Test 5: ECMP Load Balancing - Packets distributed between S2 and S3 (4 points)
min_expected_ratio = 0.30
max_expected_ratio = 0.70

s2_pkts = len(s2_packets)
s3_pkts = len(s3_packets)

tot = 2*num_packets
s2_percentage = (s2_pkts / tot) * 100
s3_percentage = (s3_pkts / tot) * 100

condition = min_expected_ratio <= s2_percentage/100 <= max_expected_ratio and min_expected_ratio <= s3_percentage/100 <= max_expected_ratio
result5 = getResult(condition)
print(f"\n{result5} Test 5: ECMP Load Balancing - Packets distributed between S2 ({s2_percentage:.1f}%) and S3 ({s3_percentage:.1f}%), Expected: [{(min_expected_ratio*100):.0f}% - {(max_expected_ratio*100):.0f}%] / [{(max_expected_ratio*100):.0f}% - {(min_expected_ratio*100):.0f}%] split between S2 and S3 switches (4 points)")
if verbose:
    print(f" \tPackets captured on S2: {s2_pkts}/{tot} ({s2_percentage:.1f}%)")
    print(f" \tPackets captured on S3: {s3_pkts}/{tot} ({s3_percentage:.1f}%)")
assert condition, f"Test 5 failed: Packets not evenly distributed between S2 ({s2_percentage:.1f}%) and S3 ({s3_percentage:.1f}%)"

points_from_b = 3 if result5 == '✅ PASS |' else 0

# Print the total points earned
print(f"\n➡️ Total points earned: {points_earned + points_from_b}/{total_points}")

---
## Notes on Debugging and Troubleshooting

- Run the cells below "Custom Header Tunnel Forward Test" to set up the test environment.

- Open the required number of terminals for your topology. You'll need one terminal for each switch and host.

- Copy the SSH commands provided in the output and paste them into the corresponding terminals to establish SSH connections to each node.

- Verify that the `s<n>.p4` code file exists on each switch, where `<n>` represents the switch number. Also, check for the presence of the corresponding `add_rules_s<n>.sh` files.

- In each switch terminal, run the command `tail -f logs` to monitor the logs in real-time as traffic is being processed. This will help you observe the behavior of the switches and detect any issues.

- If you encounter any problems, carefully review the `s<n>.p4` code for each switch to ensure that the custom header definition, parsing, and processing logic are correctly implemented.

- Verify that the table entries and actions in the `add_rules_s<n>.sh` files are properly defined and match the expected behavior of your custom tunneling protocol.

- Pay attention to the logs generated by the switches during traffic replay. Look for any error messages, unexpected behavior, or deviations from the expected forwarding logic.

- If necessary, add additional log statements in the `s<n>.p4` code to help track the flow of packets and identify the specific points where issues may be occurring. Here's an example how to add logs: 
```p4
log_msg("DEBUG: IPv4 src address {} and dest address {}.", {hdr.ipv4.srcAddr, hdr.ipv4.dstAddr});
```

- If creating a slice takes too long (more than 20 mins) or the slice provosioning fails continuously, pass a `site="<SITE>"` param to the `.add_node()` methods, where `"<SITE>"` is any available sites on FABRIC. Make sure to select sites that are not under maintainance. You can take a list of available sites and its resources availability on the home page of the [FABRIC portal](https://portal.fabric-testbed.net/).

- Ignore warnings such as below
```bash
Warning: Permanently added 'bastion.fabric-testbed.net' (ED25519) to the list of known hosts.
Warning: Permanently added '2610:148:1f00:9f01:f816:3eff:fefc:864d' (ED25519) to the list of known hosts.
```

## Useful Resources

- [FabricTestbed-Extensions Documentation](https://learn.fabric-testbed.net/docs/fablib/genindex.html)
- [P4 Cheatsheet](https://drive.google.com/file/d/1Z8woKyElFAOP6bMd8tRa_Q4SA1cd_Uva/view)
- [P416 Portable Switch Architecture (PSA)](https://p4.org/p4-spec/docs/PSA-v1.0.0.html)
- [P4 Labs (BMv2)](./fabric_examples/complex_recipes/p4_labs_bmv2/main.ipynb)

### Submission

To create a `.tar.gz` file that compresses all the required files, run the provided cell in your notebook. This will generate a compressed `.tar.gz` file in the same folder. Download the generated `.tar.gz` file to your local disk and **upload this file to Blackboard as your final submission.**

In [ ]:
%%sh

# Read the username from the *_assignment5.txt file
username=$(ls *_assignment5.txt | cut -d '_' -f 1)

# Create a tar.gz file using the username variable
tar -czf "${username}.tar.gz" --exclude="*_assignment5.txt" --exclude="*.tar.gz" *

If the above doesn't work feel free to create the tar.gz on your own via the terminal using the `tar -czf` command.